# IMP npctranport Module Tutorial
### Written by: Roi Eliasian roi.eliasian@mail.huji.ac.il ; Barak Raveh barak.raveh@mail.huji.ac.il 
### Last updated: Feb 21st, 2025
This tutorial provides a step-by-step guide to executing an NPC transport simulation using the `npctransport` module, and analyzing the results.

Before proceeding, ensure that you are running this notebook in a Python environment with the `IMP`, `NumPy` and `matplotlib` libraries installed.
See https://integrativemodeling.org/nightly/doc/manual/installation.html for installation instructions, including on Google Colab.


## 1. Configuration Files

Before running the simulation, a configuration file must be generated. This file defines the components of the simulation (e.g., FG Nups, NTRs) and their interactions.

The following code generates a sample configuration file describing a simplified pore structure. This pore consists solely of NSP1 Nups and follows a basic toroidal scaffold shape. Additionally, a number of NTRs are included.

For details on the configration parameters, please refer to the simulation manual.

In [ ]:
#!/usr/bin/python
from IMP.npctransport import *
import itertools
import numpy as np

outfile = "config.pb"

config = Configuration()
IMP.npctransport.set_default_configuration(config)
config.statistics_fraction.lower = 1.0
config.interaction_k.lower = 1e-09
config.interaction_range.lower = 10
config.backbone_k.lower = 0.0075
config.time_step_factor.lower = 2.0
config.slack.lower = 10
config.number_of_trials = 1
config.statistics_interval_ns = 0.1
config.dump_interval_ns = 1
config.output_statistics_interval_ns = 1
config.simulation_time_ns = 1000
config.box_is_on.lower = 1
config.box_side.lower = 1500
config.slab_is_on.lower = 2
config.slab_thickness.lower = 150
config.tunnel_radius.lower = 185
config.is_xyz_hist_stats = 1
config.backbone_tau_ns.lower = 50.0
config.is_backbone_harmonic = 1
config.output_npctransport_version = 4.5
config.xyz_stats_crop_factor = 1
config.xyz_stats_voxel_size_a = 10
config.xyz_stats_max_box_size_a = 700
config.nonspecific_k.lower = 0.01
config.excluded_volume_k.lower = 10.0
config.angular_D_factor.lower=0.3
config.is_multiple_hdf5s = True
config.full_output_statistics_interval_factor = 100


anchor_coordinates = [[185.0, 0.0, -75.0], [130.8147545195113, 130.8147545195113, -75.0], [1.1327982892113017e-14, 185.0, -75.0], [-130.81475451951127, 130.8147545195113, -75.0], [-185.0, 2.2655965784226034e-14, -75.0], [-130.81475451951133, -130.81475451951127, -75.0], [-3.398394867633905e-14, -185.0, -75.0], [130.81475451951127, -130.81475451951133, -75.0], [120.0480947161671, 0.0, -37.5], [84.88682184232671, 84.88682184232671, -37.5], [7.350825746894615e-15, 120.0480947161671, -37.5], [-84.8868218423267, 84.88682184232671, -37.5], [-120.0480947161671, 1.470165149378923e-14, -37.5], [-84.88682184232673, -84.8868218423267, -37.5], [-2.2052477240683848e-14, -120.0480947161671, -37.5], [84.88682184232668, -84.88682184232673, -37.5], [120.0480947161671, 0.0, 37.499999999999986], [84.88682184232671, 84.88682184232671, 37.499999999999986], [7.350825746894615e-15, 120.0480947161671, 37.499999999999986], [-84.8868218423267, 84.88682184232671, 37.499999999999986], [-120.0480947161671, 1.470165149378923e-14, 37.499999999999986], [-84.88682184232673, -84.8868218423267, 37.499999999999986], [-2.2052477240683848e-14, -120.0480947161671, 37.499999999999986], [84.88682184232668, -84.88682184232673, 37.499999999999986], [185.0, 0.0, 75.0], [130.8147545195113, 130.8147545195113, 75.0], [1.1327982892113017e-14, 185.0, 75.0], [-130.81475451951127, 130.8147545195113, 75.0], [-185.0, 2.2655965784226034e-14, 75.0], [-130.81475451951133, -130.81475451951127, 75.0], [-3.398394867633905e-14, -185.0, 75.0], [130.81475451951127, -130.81475451951133, 75.0]]
suffix_list = ["_C"] * 12 + ["_N"] * 4
#lowered the amount of beads proportinaly and same with the type: 23,9 -> 12,4

############
# Add FGs: #
############

fgs = []
for frame_i, coordinates in enumerate(anchor_coordinates):
    cur_fg = IMP.npctransport.add_fg_type(config,
                                          type_name=f"fg{frame_i}",
                                          number_of_beads=16,
                                          number=1,
                                          radius=8.0,
                                          interactions=1,
                                          rest_length_factor=1.9,
                                          d_factor=1.0,
                                          interaction_k_factor=1.0,
                                          interaction_range_factor=1.0)
    pos = cur_fg.anchor_coordinates.add()
    pos.x = coordinates[0]
    pos.y = coordinates[1]
    pos.z = coordinates[2]
    fgs.append(cur_fg)
    cur_fg.type_suffix_list.extend(suffix_list)

#################################
# add internal fg cohesiveness: #
#################################
self_k_N = 1.47
self_k_C = 1.32
self_range = 6.00
self_nonspec_k_N = 0.01
self_nonspec_k_C = 0.08
for frame_i in itertools.combinations(range(len(fgs)), 2):
    for suffix0 in ["_N", "_C"]:
        for suffix1 in ["_N", "_C"]:
            interactionFG_FG = IMP.npctransport.add_interaction(config,
                                                        name0=f"fg{frame_i[0]}{suffix0}",
                                                        name1=f"fg{frame_i[1]}{suffix1}",
                                                        interaction_k=0.5*(self_k_N if suffix0=="_N" else self_k_C) + 0.5*(self_k_N if suffix1=="_N" else self_k_C),
                                                        interaction_range=self_range)
            interactionFG_FG.nonspecific_k.lower = np.sqrt((self_nonspec_k_N if suffix0=="_N" else self_nonspec_k_C) * (self_nonspec_k_N if suffix1=="_N" else self_nonspec_k_C))

for frame_i in range(len(fgs)):
    for suffix0 in ["_N", "_C"]:
        for suffix1 in ["_N", "_C"]:
            interactionFG_FG = IMP.npctransport.add_interaction(config,
                                                        name0=f"fg{frame_i}{suffix0}",
                                                        name1=f"fg{frame_i}{suffix1}",
                                                        interaction_k=0.5*(self_k_N if suffix0=="_N" else self_k_C) + 0.5*(self_k_N if suffix1=="_N" else self_k_C),
                                                        interaction_range=self_range)
            interactionFG_FG.nonspecific_k.lower = np.sqrt((self_nonspec_k_N if suffix0=="_N" else self_nonspec_k_C) * (self_nonspec_k_N if suffix1=="_N" else self_nonspec_k_C))


#############
# Add NTRs: #
#############

ntr_vals = [(30,5)] # Radius 30, 5 Interaction sites distributed uniformally
ntrs = []
for vals in ntr_vals:
    cur_ntr  = IMP.npctransport.add_float_type(config,
                                number=400,
                                radius=vals[0],
                                type_name=f"NTR{vals[0]}",
                                interactions=vals[1],
                                d_factor=1.0,
                                interaction_k_factor=1.0,
                                interaction_range_factor=1.0)
    ntrs.append(cur_ntr)

############################
# Add NTR-FG interactions: #
############################
for frame_i in range(len(fgs)):
    for vals in ntr_vals:
        for suffix in ["_N", "_C"]:
            IMP.npctransport.add_interaction(config,
                                     name0=f"fg{frame_i}{suffix}",
                                     name1=f"NTR{vals[0]}",
                                     interaction_k=2.64,
                                     interaction_range=5.5,
                                     range_sigma0_deg=45.0,
                                     range_sigma1_deg=45.0)

# dump to file
f = open(outfile, "wb")
f.write(config.SerializeToString())

## 2. Running the Simulation
With the configuration file generated, we are now ready to run the simulation. The configuration file is provided to the `fg_simulation` executable, found in the IMP installation, via the `--configuration` flag.

For details on additional flags that can be passed to the simulation executable, please refer to the simulation manual.

In [ ]:
%%bash
VENV_FOLDER=... # Insert path to the python virtual environment where IMP was installed
CONFIG_PATH=config.pb # From previous section
OUTPUT_PATH=output #output prefix

${VENV_FOLDER}/bin/fg_simulation --configuration ${CONFIG_PATH} --output ${OUTPUT_PATH}.pb --conformations ${OUTPUT_PATH}$.movie.rmf --final_conformations ${OUTPUT_PATH}.final.rmf --short_init_factor 0.5 --short_sim_factor 1.00 

## 3. Processing simulation output

The command above will execute the simulation, running for 1000 ns as specified in the previously generated configuration file. Upon completion, the simulation will produce three output files:
- `output.pb` – Contains various statistical data about the simulation. This file can be processed using the IMP library in Python (see Section 3.1).
- `output.movie.rmf` – Stores molecular trajectory data for the entire simulation, following the RMF format. The framerate is determined by the `output_statistics_interval_ns` parameter in the configuration file.
- `output.final.rmf` – Contains only the final conformations from the simulation.

The RMF files adhere to the format described [here](https://integrativemodeling.org/rmf/format.html). These trajectory files can be visualized using [UCSF Chimera](https://www.cgl.ucsf.edu/chimera/) or manually processed with the IMP library (see Section 3.2).


### 3.1 Processing output.pb
The following plots a measure of the free energy of the simulation over time. (Just one of the many statistics that the simulation computes and saves in the `pb` format. A full list can be found here [here](https://github.com/salilab/npctransport/blob/b411ecb0a260d99a8e15e7fbb080cadc2efd362a/data/npctransport.proto)).

In [ ]:
import IMP.npctransport
import matplotlib.pyplot as plt

def load_time_energy(pb_path):
    with open(pb_path, "rb") as f:
        output = IMP.npctransport.Output()
        fstring = f.read()
        output.ParseFromString(fstring)
        time_ns = []
        energy = []
        for i in range(len(output.statistics.global_order_params)):
            time_ns.append(output.statistics.global_order_params[i].time_ns)
            energy.append(output.statistics.global_order_params[i].energy)
    return time_ns, energy

def plot_energy(time, energy):
    plt.plot(time, energy)
    plt.xlabel("Time (ns)")
    plt.ylabel("Energy (kcal/mol)")
    plt.show()
    
time, energy = load_time_energy("output.pb")
plot_energy(time, energy)

### 3.2 Processing RMF trajectories
The following code reads the trajectories of the NTRs in `output.final.rmf` and converts them to a simple numpy array of the shape \[n_NTRs, 3, n_frames(which is 1)\] 

In [ ]:
import RMF
import numpy as np

def _has_depth_with_site(root, i):
    """ returns true if node subtree thru first child is at least i
        levels, including the root node itself, and the lead is a site """
#  print root, i, len(root.get_children())
    if (i==1) and root.get_name()=="site":
        return True
    c = root.get_children()
    if len(c) == 0:
        return False
    return _has_depth_with_site(c[0], i-1)

def _add_nodes(node, tf, type_prefixes, depth=0):
    '''
    node - rmf node to scan
    tf - typed factory
    type_prefixes - list of full type prefixes (e.g. "Nup1" for "Nup1N")

    adds only nodes whose type name begins with any of the specified type prefixes
    '''
    children = node.get_children()
    ret = []
    if len(children)==0:
        return ret
    if _has_depth_with_site(node, 3) and tf.get_is(children[0]):
        child_type = tf.get(children[0]).get_type_name()
        if any([child_type.startswith(tp) for tp in type_prefixes]):
            ret.append(children)
    for c in children:
        ret += _add_nodes(c, tf,  type_prefixes, depth+1)
    return ret

def _add_nodes_exact(node, tf, types, depth=0):
    '''
    node - rmf node to scan
    tf - typed factory
    types - list of full types (e.g. "Nup1N")

    adds only nodes whose type name begins with any of the specified type prefixes
    '''
    children = node.get_children()
    ret = []
    if len(children)==0:
        return ret
    if _has_depth_with_site(node, 3) and tf.get_is(children[0]):
        child_type = tf.get(children[0]).get_type_name()
        if any([child_type == tp for tp in types]):
            ret.append(children)
    for c in children:
        ret += _add_nodes_exact(c, tf,  types, depth+1)
    return ret

def load_NTR_data(rmf_path, NTR_amount, frames_per_file=1):
    trajectories = np.zeros(shape=[NTR_amount, 3, frames_per_file])

    in_fh = RMF.open_rmf_file_read_only(f"{rmf_path}/{rmf_t}.movie.rmf")
    rff = RMF.ReferenceFrameFactory(in_fh)
    tf = RMF.TypedFactory(in_fh)
    NTR_types = [f"NTR30"] # name defined in config.pb

    # load data
    type2chains={}
    for NTR_type in NTR_types:
        type2chains[NTR_type] = _add_nodes(in_fh.get_root_node(), tf, [NTR_type])
    frame_i = 0
    for f_id, f in enumerate(in_fh.get_frames()):
        in_fh.set_current_frame(f)

        # read data
        for NTR_i in range(NTR_amount):
            coord = rff.get(type2chains["NTR30"][0][NTR_i]).get_translation()
            
            trajectories[NTR_i, 0, frame_i] = coord[0] / 10 # In nm
            trajectories[NTR_i, 1, frame_i] = coord[1] / 10
            trajectories[NTR_i, 2, frame_i] = coord[2] / 10
        if frame_i >= frames_per_file:
            break
        frame_i += 1
        
    return trajectories

trajectories = load_NTR_data("output.final.rmf", 400, 1)